In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from preprocessing import *

In [2]:
train_file_ds = tf.data.Dataset.list_files(['complete_training_data/*.wav'])
validation_file_ds = tf.data.Dataset.list_files(['complete_validation_data/*.wav'])

batch_size = 5

train_mel_ds = train_file_ds.map(get_log_mel_spectrogram)
for spectrogram, label in train_mel_ds.take(1):
    SHAPE = spectrogram.shape

def preprocess_with_resized_mel(filename):
    log_mel_spectrogram, label = get_log_mel_spectrogram(filename)
    log_mel_spectrogram.set_shape(SHAPE)
    mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)[..., :]
    mfccs = tf.expand_dims(mfccs, -1)
    mfccs = tf.image.resize(mfccs, [32, 32])
    label_id = tf.argmax(label == LABELS)

    return mfccs, label_id

# train_file_ds = tf.data.Dataset.list_files(["complete_training_data/*.wav"])
train_ds = train_file_ds.map(preprocess_with_resized_mel).batch(batch_size)
validation_ds = validation_file_ds.map(preprocess_with_resized_mel).batch(batch_size)

for example_batch, example_labels in train_ds.take(1):
    print(example_batch.shape)
    print(example_labels)

(5, 32, 32, 1)
tf.Tensor([2 2 2 1 2], shape=(5,), dtype=int64)


In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
    ])

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.optimizers.Adam(learning_rate=1e-5)
metrics = [tf.metrics.SparseCategoricalAccuracy()]

In [4]:
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
history = model.fit(train_ds, epochs=5, validation_data=validation_ds)
test_loss, test_accuracy = model.evaluate(validation_ds)

Epoch 1/5
75/75 [==============================] - 32s 402ms/step - loss: 1.0833 - sparse_categorical_accuracy: 0.4305 - val_loss: 1.1762 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/5
75/75 [==============================] - 30s 396ms/step - loss: 0.9963 - sparse_categorical_accuracy: 0.4305 - val_loss: 1.2376 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/5
75/75 [==============================] - 30s 396ms/step - loss: 0.9686 - sparse_categorical_accuracy: 0.4358 - val_loss: 1.2933 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/5
75/75 [==============================] - 34s 457ms/step - loss: 0.9550 - sparse_categorical_accuracy: 0.5000 - val_loss: 1.3348 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/5
6/6 [==============================] - 3s 445ms/step - loss: 1.3796 - sparse_categorical_accuracy: 0.0000e+00
